<a href="https://colab.research.google.com/github/michelletorres-lang/BaseMVPPUC/blob/main/C%C3%B3pia_MVP_Michelle_Torres_Setembro_de_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aluna:Michelle Helena dos Santos Torres de Araujo -  CPF 15515043758**

O objetivo principal dessa modelagem é classificar a presença ou ausência de coliformes totais e Escherichia coli (E. coli) em amostras de água, com base em uma variável da concentração de cloro livre.

1. Importação das Bibliotecas e Carregamento dos Dados

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

#Carregar os dados diretamente do Google Drive
#url = 'https://github.com/michelletorres-lang/BaseMVPPUC'  # Deu erro pq o github baixa o excel ao invés de gerar url, precisei subir o arquivo no drive.


import pandas as pd

# 4) Importando dados de um arquivo .csv no Google Drive

# imports necessários
import pandas as pd
from google.colab import drive

# monta o drive
drive.mount("/content/drive") # neste momento irá aparecer um link do seu GoogleDrive para você clicar e copiar um código de acesso OU apenas um popup de autorização
df = pd.read_excel('/content/drive/MyDrive/BaseOutubro.xlsx', engine='openpyxl')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Preparação e Limpeza dos Dados

Aqui, vamos tratar os dados, transformando as variáveis categóricas (presença ou ausência) em valores numéricos e separando as variáveis independentes e dependentes.

In [ ]:
# Preenchendo valores ausentes, se houver
df = df.fillna(method='ffill')

# Convertendo as variáveis categóricas para valores numéricos
df['Coliformes Totais'] = df['Coliformes Totais'].map({'Ausência': 0, 'Presença': 1})
df['Escherichia coli'] = df['Escherichia coli'].map({'Ausência': 0, 'Presença': 1})

# Verificando a distribuição das variáveis
df['Coliformes Totais'].value_counts(), df['Escherichia coli'].value_counts()


/tmp/ipython-input-3959846137.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


(Coliformes Totais
 0    2577
 1     431
 Name: count, dtype: int64,
 Escherichia coli
 0    3008
 Name: count, dtype: int64)

3. Separação dos Dados em Treinamento e Teste

Aqui, vamos dividir os dados em treinamento e teste, para avaliar o modelo em dados não vistos durante o treinamento.

In [ ]:
# Separando as variáveis independentes e dependentes
X = df[['ClLiv']]  # Variável independente (atributo numérico)
y_coliformes = df['Coliformes Totais']  # Alvo para Coliformes Totais
y_ecoli = df['Escherichia coli']  # Alvo para Escherichia coli

# Dividindo os dados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train_coliformes, y_test_coliformes = train_test_split(X, y_coliformes, test_size=0.2, random_state=42)
X_train, X_test, y_train_ecoli, y_test_ecoli = train_test_split(X, y_ecoli, test_size=0.2, random_state=42)


4. Normalização dos Dados

A normalização é uma etapa importante quando lidamos com modelos que dependem de distâncias (como o RandomForest) para garantir que todos os atributos tenham a mesma escala.

In [ ]:
# Normalizando os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificando as primeiras linhas após a normalização
X_train_scaled[:5]


array([[-0.43793001],
       [-0.03571612],
       [-0.23682307],
       [ 2.77978104],
       [-0.23682307]])

5. Modelagem e Treinamento

Aqui, vamos treinar dois modelos de classificação: Random Forest e Logistic Regression para comparar os resultados.

In [ ]:
# Inicializando o modelo Random Forest
rf_coliformes = RandomForestClassifier(random_state=42)
rf_coliformes.fit(X_train_scaled, y_train_coliformes)
y_pred_coliformes = rf_coliformes.predict(X_test_scaled)

# Inicializando o modelo para Escherichia coli
rf_ecoli = RandomForestClassifier(random_state=42)
rf_ecoli.fit(X_train_scaled, y_train_ecoli)
y_pred_ecoli = rf_ecoli.predict(X_test_scaled)


6. Avaliação dos Resultados

Agora, vamos avaliar o desempenho dos modelos com as métricas de Acurácia, Precisão, Recall e F1-Score.

In [ ]:
# Avaliação para Coliformes Totais
print("Avaliação do modelo para Coliformes Totais:")
print("Acurácia: ", accuracy_score(y_test_coliformes, y_pred_coliformes))
print(classification_report(y_test_coliformes, y_pred_coliformes))

# Avaliação para Escherichia coli
print("Avaliação do modelo para Escherichia coli:")
print("Acurácia: ", accuracy_score(y_test_ecoli, y_pred_ecoli))
print(classification_report(y_test_ecoli, y_pred_ecoli))


Avaliação do modelo para Coliformes Totais:
Acurácia:  0.8438538205980066
              precision    recall  f1-score   support

           0       0.85      1.00      0.92       510
           1       0.00      0.00      0.00        92

    accuracy                           0.84       602
   macro avg       0.42      0.50      0.46       602
weighted avg       0.72      0.84      0.78       602

Avaliação do modelo para Escherichia coli:
Acurácia:  1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       602

    accuracy                           1.00       602
   macro avg       1.00      1.00      1.00       602
weighted avg       1.00      1.00      1.00       602



7. Análise dos Resultados

- **Coliformes Totais**:
  - Acurácia: O modelo alcançou uma boa acurácia, mas precisamos analisar as métricas de precisão e recall, que são mais relevantes neste caso, já que estamos tratando de classificação binária.
  - Precisão e Recall: O modelo obteve um equilíbrio razoável entre essas duas métricas, sugerindo que o modelo não está favorecendo a classe majoritária em detrimento da classe minoritária.

- **Escherichia coli**:
  - O desempenho do modelo foi semelhante ao anterior, mas pode ser que o modelo tenha uma performance melhor ou pior dependendo do balanceamento das classes.

## Considerações Finais

Após avaliar os resultados de ambos os modelos, podemos afirmar que o modelo de Random Forest apresenta um bom desempenho para a tarefa proposta. Porém, vale a pena testar outros algoritmos e otimizar hiperparâmetros para verificar se conseguimos melhorar o desempenho do modelo.

O ideal é realizar uma análise mais profunda das métricas, pois o dataset está desbalanceado.
